In [432]:
import re
import string
import nltk
import tensorflow_datasets as tfds
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from string import punctuation
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from bs4 import BeautifulSoup
from tqdm import tqdm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [433]:
# Load the IMDb reviews dataset
data = tfds.load('imdb_reviews', split={'train': 'train', 'test': 'test'})

In [434]:
# Convert the data into pandas DataFrame and decode bytes to string
train_df = tfds.as_dataframe(data['train'])
test_df = tfds.as_dataframe(data['test'])

train_df['text'] = train_df['text'].apply(lambda x: x.decode('utf-8'))
test_df['text'] = test_df['text'].apply(lambda x: x.decode('utf-8'))

train_df['label'] = train_df['label'].replace({0: 'negative', 1: 'positive'})
test_df['label'] = test_df['label'].replace({0: 'negative', 1: 'positive'})

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train_df, train_df['label'], test_size=0.2, random_state=42, stratify=train_df['label'])

In [435]:
# Label encoding for the sentiment labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [436]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(20000, 2) (20000,)
(5000, 2) (5000,)


# Data Cleaning

In [437]:
spell = SpellChecker()
lemmatizer = WordNetLemmatizer()


def transformations(dataframe):
    # HTML Tags removal
    dataframe['text'] = dataframe['text'].apply(lambda words: re.sub('<[^<]+?>', '', words))

    # Lower case conversion
    dataframe['text'] = dataframe['text'].apply(lambda words: words.lower())

    # Word Tokenization
    dataframe['text'] = dataframe['text'].apply(word_tokenize)

    # Punctuation removal
    dataframe['text'] = dataframe['text'].apply(lambda words: [x for x in words if not x in punctuation])

    # Number removal
    dataframe['text'] = dataframe['text'].apply(lambda words: [x for x in words if not x.isdigit()])

    # Spellchecker and remove none values
    dataframe['text'] = dataframe['text'].apply(lambda words: [spell.correction(x) for x in words])
    dataframe['text'] = dataframe['text'].apply(lambda words: [x for x in words if x is not None])

    # Stopword removal
    dataframe['text'] = dataframe['text'].apply(lambda words: [x for x in words if x not in stopwords.words('english')])

    # Lemmatization
    dataframe['text'] = dataframe['text'].apply(lambda words: [lemmatizer.lemmatize(x) for x in words])

    # Join again
    dataframe['text'] = dataframe['text'].apply(lambda words: " ".join(words))

    return dataframe

In [438]:
x_train = x_train[:5]
x_test = x_test[:5]

In [439]:
clean_data_train_data = tqdm(transformations(x_train))
x_train['cleaned_text'] = clean_data_train_data

 40%|████      | 2/5 [00:00<?, ?it/s]


In [440]:
clean_data_test_data = tqdm(transformations(x_test))
x_test['cleaned_text'] = clean_data_test_data

 40%|████      | 2/5 [00:00<00:00, 1998.72it/s]


# Vectorizer

In [441]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec = vec.fit(x_train['cleaned_text'])
train_x_bow = vec.transform(x_train['cleaned_text'])
test_x_bow = vec.transform(x_test.cleaned_text)

In [442]:
print(train_x_bow.shape)
print(test_x_bow.shape)

(2, 2)
(2, 2)


# Naive Bayes with Hyperparameter Tuning

In [443]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

In [444]:
classifier = MultinomialNB()

In [445]:
alpha_ranges = {
    "alpha": [0.001, 0.01, 0.1, 1, 10.0, 100]
}

In [446]:
grid_search = GridSearchCV(classifier, param_grid=alpha_ranges, scoring='accuracy', cv=3, return_train_score=True)
grid_search.fit(train_x_bow, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [2, 20000]

In [ ]:
alpha = [0.001, 0.01, 0.1, 1, 10.0, 100]
train_acc = grid_search.cv_results_['mean_train_score']
train_std = grid_search.cv_results_['std_train_score']

test_acc = grid_search.cv_results_['mean_test_score']
test_std = grid_search.cv_results_['std_test_score']

In [ ]:
import matplotlib.pyplot as plt

plt.plot(alpha, train_acc, label="Training Score", color='b')
plt.plot(alpha, test_acc, label="Cross Validation Score", color='r')

plt.title("Validation Curve with Naive Bayes Classifier")
plt.xlabel("Alpha")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.legend(loc = 'best')
plt.show()


In [ ]:
grid_search.best_estimator_

classifier = MultinomialNB(alpha=1)
classifier.fit(train_x_bow, y_train)

In [ ]:
predict = classifier.predict(test_x_bow)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy is ", accuracy_score(y_test, predict))

In [ ]:
from sklearn.metrics import classification_report
print("Accuracy is ", classification_report(y_test, predict))